In [ ]:
import torch
import numpy as np
from sklearn.metrics import precision_recall_curve ,average_precision_score
from sklearn.preprocessing import label_binarize
 
recall,precision,_=np.load("pr.npy",allow_pickle=True)
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
# 绘制每个类别的 PR 曲线
for i in range(10):
    
    plt.plot(recall[i], precision[i], label=f'Class {i}')
plt.show()

In [ ]:
#单文件转换 
import json,os
import numpy as np
def allfile(path):
    fileall=[]
    for curpath,dirlist,filelist in os.walk(path):
        for i in filelist:
            fileall.append(os.path.join(curpath, i))
             
            
    return fileall
root='/home/dongyiting/datasets/pose_estimation/'

fileall=allfile(root)
count=0
idx=0
# print((fileall[0]))
for file in fileall:

    
    # 读取JSON文件
    with open(file, 'r') as f:
        data = json.load(f)

    path=file[len(root):].replace(".json","/")
     


    fulldata=dict()
    fulldata['images']=[]
    fulldata['annotations']=[]
    fulldata['categories']=[ {'id': 1, 'name': 'person'}]
    
    # print(data)
    dictkey={}

    alldd=[]
    for d in data:

        name=d["image_id"]
        height=260
        width=346
        id=int(d["image_id"].replace(".jpg","")) 

        if id not in dictkey:
            dictkey[id]=id
            fulldata['images'].append( {"file_name":path+name, "height":height, "width":width,"id":id} )
 
        dataann=d
        dataann['segmentation']=[[0]]
        dataann['num_keypoints']=len(d["keypoints"])//3


        ll=[i for i in range(26*3) if i%3==2]
        
        dd=np.array(dataann["keypoints"])
        dd[np.logical_and( dd>0.2 , dd<1)] =2
        dd[np.logical_and( dd<0.2 , dd<1)] =0
        dd=dd.astype(np.int16)
        dataann["keypoints"]=dd.tolist()
        dataann["image_id"]=id
        # print(dataann["keypoints"])
       
        # alldd.append(dd[ll])

        

        dataann['area']=d['box'][-1]*d['box'][-2]
        dataann['bbox']=d['box'] 
        dataann['iscrowd']=0
        dataann['id']=count
        count+=1
        fulldata['annotations'].append(dataann)
    path=file.replace("pose_estimation","pose_estimation_coco")
    with open(path, 'w') as f:
        json.dump(fulldata, f)
         
    
    print(idx)
    idx+=1
    # if idx>1000:
    #     dd=np.histogram(alldd,bins=50,range=[0,1] )
    #     print(dd)
    #     break
 

In [10]:
#单文件转换 
import json,os
import numpy as np
def allfile(path):
    fileall=[]
    for curpath,dirlist,filelist in os.walk(path):
        for i in filelist:
            fileall.append(os.path.join(curpath, i))
             
            
    return fileall
root='/home/dongyiting/datasets/pose_estimation/'

fileall=allfile(root)
count=0
idx=0
fulldata=dict()
fulldata['images']=[]
fulldata['annotations']=[]
fulldata['categories']=[ {'id': 1, 'name': 'person'}]
val=False
# val=True

validx=np.zeros(len(fileall),dtype=bool)
validx[range(0,len(fileall),5)]=1

if val:fileall=np.array(fileall)[validx]
else:fileall=np.array(fileall)[~validx]

# print((fileall[0]))
for file in fileall:

  
    # 读取JSON文件
    with open(file, 'r') as f:
        data = json.load(f)

    path=file[len(root):].replace(".json","/")
     
    # 读取JSON文件
    length=len(allfile("/data/datasets/Bullying10k_eventimage/"+path))

    # print(len(allfile("/data/datasets/Bullying10k_eventimage/"+path)))
    # continue
    # print(data)
    dictkey={}

    alldd=[]
    for d in data:

        name=d["image_id"].replace(".jpg",".png")
        height=260
        width=346
        id=idx*1000+int(d["image_id"].replace(".jpg","")) 
        if int(d["image_id"].replace(".jpg",""))>=length:
            break
        if id not in dictkey:
            dictkey[id]=id
            fulldata['images'].append( {"file_name":path+name, "height":height, "width":width,"id":id} )
 
        dataann=d
        dataann['segmentation']=[[0]]
        dataann['num_keypoints']=len(d["keypoints"])//3


        ll=[i for i in range(26*3) if i%3==2]
        
        dd=np.array(dataann["keypoints"])
        dd[np.logical_and( dd>0.2 , dd<1)] =2
        dd[np.logical_and( dd<0.2 , dd<1)] =0
        dd=dd.astype(np.int16)
        dataann["keypoints"]=dd.tolist()
        dataann["image_id"]=id

        # print(dataann )
        # break
       
        # alldd.append(dd[ll])

        

        dataann['area']=d['box'][-1]*d['box'][-2]
        dataann['bbox']=d['box'] 
        dataann['iscrowd']=0
        dataann['id']=count
        count+=1
        fulldata['annotations'].append(dataann)
    # path=file.replace("pose_estimation","pose_estimation_coco")
    # with open(path, 'w') as f:
    #     json.dump(fulldata, f)
         
    # break
    # print(idx)
    idx+=1
if val:
    with open("/data/datasets/Bullying10k_eventimage/val_keypoints.json", 'w') as f:
        json.dump(fulldata, f)
else:
    with open("/data/datasets/Bullying10k_eventimage/train_keypoints.json", 'w') as f:
        json.dump(fulldata, f)
    # if idx>1000:
     
    #     print(fulldata["images"][0])
    #     break
 